# Projet Deep Learning

In [2]:
import os
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import seaborn as sns

from IPython.display import display, HTML

In [25]:
titre = ["sequence","structure","predicted_loop_type", "seq_length", "seq_scored"]
data = pd.read_json('test.json', lines = True)

print(data)
X = data[titre]

print(X)

#print(data)

      index            id                                           sequence  \
0         0  id_00073f8be  GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...   
1         1  id_000ae4237  GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...   
2         2  id_00131c573  GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...   
3         3  id_00181fd34  GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...   
4         4  id_0020473f7  GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...   
...     ...           ...                                                ...   
3629   3629  id_ff691b7e5  GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...   
3630   3630  id_ff9bf3581  GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...   
3631   3631  id_ffc8f96a8  GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...   
3632   3632  id_ffd7e8cc1  GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...   
3633   3633  id_ffda94f24  GGAAAAACCAAUUGCUAUUAAAGGUUCCUUUGUUCGCUAAGUCCAA...   

                                       

In [10]:
#print(data)

In [4]:
print(len(X.loc[0, 'predicted_loop_type']))

107


In [37]:
i=1
seq = list(X.loc[i,"sequence"])
print(seq)

['G', 'G', 'A', 'A', 'A', 'C', 'G', 'G', 'G', 'U', 'U', 'C', 'C', 'G', 'C', 'G', 'G', 'A', 'U', 'U', 'G', 'C', 'U', 'G', 'C', 'U', 'A', 'A', 'U', 'A', 'A', 'G', 'A', 'G', 'U', 'A', 'A', 'U', 'C', 'U', 'C', 'U', 'A', 'A', 'A', 'U', 'G', 'C', 'A', 'G', 'C', 'U', 'A', 'C', 'C', 'G', 'G', 'C', 'U', 'C', 'U', 'A', 'U', 'A', 'A', 'U', 'G', 'A', 'G', 'C', 'A', 'A', 'A', 'A', 'A', 'C', 'G', 'G', 'U', 'A', 'A', 'A', 'U', 'C', 'C', 'C', 'G', 'A', 'C', 'A', 'A', 'G', 'C', 'U', 'U', 'G', 'A', 'U', 'U', 'U', 'C', 'G', 'A', 'U', 'C', 'A', 'A', 'G', 'C', 'A', 'A', 'A', 'A', 'G', 'A', 'A', 'A', 'C', 'A', 'A', 'C', 'A', 'A', 'C', 'A', 'A', 'C', 'A', 'A', 'C']


In [38]:
#a = np.resize(np.arange(0),(4,7,3,130,3634))

a = np.resize(np.arange(0),(3634,130,3,7))

for i in range(len(X)) :
    seq = list(X.loc[i,"sequence"])
    struct = list(X.loc[i,"structure"])
    pred = list(X.loc[i,"predicted_loop_type"])
    for j in range(len(X.loc[i,"structure"])) :
        if seq[j] == 'A':
            a[i,j,1,0] = 1
        elif seq[j] == 'U':
            a[i,j,1,1] = 1
        elif seq[j] == 'C':
            a[i,j,1,2] = 1
        elif seq[j] == 'G':
            a[i,j,1,3] = 1
        if  struct[j] == '.':
            a[i,j,2,0] = 1
        elif  struct[j] == '(':
            a[i,j,2,1] = 1
        elif  struct[j] == ')':
            a[i,j,2,2] = 1
        if  pred[j] == 'E':
            a[i,j,0,0] = 1
        elif  pred[j] == 'S':
            a[i,j,0,1] = 1
        elif  pred[j] == 'B':
            a[i,j,0,2] = 1
        elif  pred[j] == 'H':
            a[i,j,0,3] = 1
        elif  pred[j] == 'I':
            a[i,j,0,4] = 1
        elif  pred[j] == 'M':
            a[i,j,0,5] = 1
        elif  pred[j] == 'X':
            a[i,j,0,6] = 1


print(a)

[[[[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  ...

  [[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]

  [[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]

  [[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]]


 [[[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  ...

  [[1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [0 0 1 ... 0 0 0]
   [1 0 0 ... 0 0 0]]]


 [[[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  [[1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]
   [1 0 0 ... 0 0 0]]

  ...



In [33]:
print(len(X))
print(X["structure"][4])
print(len(X["structure"]))
for i in range(len(X["structure"])) :
        print(X["structure"][i])

3634
.....(((((((((((((((((((((((((((((((((((((((((....)))))))))))))))))))))))))))))))))))))))))(((((((....))))))).....................
3634
......((((((((((.(((((.....))))))))((((((((...)))))...)))))))...))).(((((((....))))))).....................
.....((((..((((((...(((((.....((((....))))....)))))..)))((((......)))).....))).....))))....(((((((....))))))).....................
...........((.(((.(.(..((..((..((((...))))..))..))..).).))).))......(((((((....))))))).....................
......((((((((((....))))))))))((((((..((.(((....))).))..))))))......(((((((....))))))).....................
.....(((((((((((((((((((((((((((((((((((((((((....)))))))))))))))))))))))))))))))))))))))))(((((((....))))))).....................
.....(((...(((......(((((((((.((....(((.....)))))))))))))))))...))).(((((((....))))))).....(((((((....))))))).....................
........(((((((((((.((..(((((((....))))....)))...)).)))...(((...((((....))))....)))))))))))(((((((....))))))).....................
.....(((((.

.....(((.((((((((((...((..(((((.((.(((..((((....))))....))).)).)))))..)).))))).)))))....)))(((((((....))))))).....................
............................(((.((...(((((...(((((...........))))).)))))..)).)))...........(((((((....))))))).....................
........((((((((((((((((((((((......))))))))))))))))))))))(((((((((.....)))))))))..........(((((((....))))))).....................
..........(((.(((((....(((....(((((((.....)))))))....)))...))))).)))..((((.....))))........(((((((....))))))).....................
..........................((((((((((((((..................................))))))))))))))...(((((((....))))))).....................
...........(((((.((((....(((((.(((((((....(((((.........))))).....)))))))))))))))).)))))...(((((((....))))))).....................
.......((((((.((((....))))..((((....(((((....)))))...)))).....))))))(((((((....))))))).....(((((((....))))))).....................
...................................................................................

.....((((((((((((((....))..)))).))))((((.((((.(((....)))))))))))))))(((((((....))))))).....................
......(((((((((((((.(((.(((.(((.((((((.......)))..))).))).))).))).........))).))).))).)))).(((((((....))))))).....................
........(((((((((.(((....))).((((....)))).(((....))).))))...)))))...(((((((....))))))).....................
...........................................................................................(((((((....))))))).....................
.....(((((((((.(((((((...((((...)))).)))))))..))))))))).(((((((..(((((((....))))))).)))))))(((((((....))))))).....................
....................................................................(((((((....))))))).....................
.....(((((((((((((((....))))))))(((((...)))))(((((((....)))))))((((((....))))))..)))))))...(((((((....))))))).....................
.....(((((((.(...(((((((.....))))......((((.....)))))))...).))))))).(((((((....))))))).....................
.......((((.((((((((((((((((((.....((((((...

.....((((..((.((.(.(.(....).).).)).))..)))).((..((..((....))..))..))(((((((....))))))).....................
......((((((((((((((((((((((....)))))))))((((.(((((((((((...))))))))))))))))))).)))).))))).(((((((....))))))).....................
.....(((((((((((.(((((((((((..((((....((((.....))))....))))..))))))))))))))))))).))).......(((((((....))))))).....................
.......((((((((((((((((((..((((.(((((((..((((((....)))))))))))))))))..))))))))))))))))))...(((((((....))))))).....................
........(((((((((((((((((((((((....)))))))))))))))))))))))(((((((((.....)))))))))..........(((((((....))))))).....................
.....(((((((...((((((.(((..((((((((((((((.((((((...)))))).)))))))..)))))))))).)))))))))))))(((((((....))))))).....................
..........................(((.((((((((((..................................)))))))))).)))...(((((((....))))))).....................
..........((((((((((((((((((((((.....))))))))))))))))))))))................................(((((((....))))